In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

chrome_path = '/Users/Moraes/Desktop/chromedriver'

chrome_service = Service(chrome_path)

options = Options()

options.add_argument('--headless')

driver = webdriver.Chrome(service=chrome_service)

# mediante get pasamos a la url que deseamos acceder
driver.get('https://www.tripadvisor.es/Restaurants')
driver.maximize_window()

time.sleep(20)
cookies = driver.find_element(By.ID, 'onetrust-accept-btn-handler')
cookies.click()
time.sleep(2)

buscador = driver.find_element('xpath', '//*[@id="component_7"]/div/div[2]/div/form/div/div/input')
buscador.click()
buscador.send_keys('Barcelona')
time.sleep(1) # dejamos que cargue otro segundo
buscador.send_keys(Keys.ENTER) # le damos enter
time.sleep(10) 

# Extraer datos, convertir html a texto
html_txt = driver.page_source
soup = BeautifulSoup(html_txt, 'html.parser')

listaRestaurantes = []

max_paginas = 31
for pagina in range(max_paginas):
    html_txt = driver.page_source
    soup = BeautifulSoup(html_txt, 'html.parser')

    # Buscar los elementos div con la clase 'result-title'
    restaurantes = soup.find_all('div', class_='result-title')

    for restaurante in restaurantes:

        # Obtener el HTML del restaurante
        restaurante_html = str(restaurante)

        # Buscar el elemento span dentro de cada restaurante para obtener el nombre
        span = restaurante.find('span')

        if span:
            nombre = span.text.strip()  # Obtener el texto y eliminar espacios en blanco alrededor

            # Buscar el elemento div con la clase 'address-text' dentro del bucle de restaurantes
            direccion_element = restaurante.find_next('div', class_='address-text')

            # Verificar si el elemento de dirección existe antes de intentar obtener el texto
            if direccion_element:
                direccion = direccion_element.text.strip()
            else:
                direccion = "Dirección no disponible"

            # Agregar el nombre y la dirección a la lista
            listaRestaurantes.append({'Nombre': nombre, 'Dirección': direccion, 'url': restaurante_html})

    # Hacer clic en el botón de siguiente página
    try:
        boton_siguiente = driver.find_element('xpath', '//*[@id="BODY_BLOCK_JQUERY_REFLOW"]/div[2]/div/div[2]/div/div/div/div/div[1]/div/div[1]/div/div[3]/div/div[2]/div/div/div/a[2]')
        boton_siguiente.click()
        time.sleep(5)  
    except Exception as e:
        print(f'Error al hacer clic en el botón de siguiente: {e}')
        break  # Salir del bucle si no se puede hacer clic en el botón de siguiente

df_name_and_address = pd.DataFrame(listaRestaurantes)

df_name_and_address

In [ ]:
pd.set_option('display.max_colwidth', None)
df_name_and_address


,Nombre,Dirección,url
0,Barcelona Tapas,"Calle Matamoros 906, Puerto Vallarta, México","<div class=""result-title"" onclick=""widgetEvCall('handlers.openResult', event, this, '/Restaurant_Review-g150793-d1032982-Reviews-Barcelona_Tapas-Puerto_Vallarta.html', {type: 'RESTAURANTS',element: this,index: 0,section: 0,locationId: '1032982',parentId: '150793',elementType: 'title',selectedId: '1032982'});""><span>Barcelona Tapas</span></div>"
1,BelleBuon,"Travessera de Gràcia 441, Barcelona, Cataluña, España","<div class=""result-title"" onclick=""widgetEvCall('handlers.openResult', event, this, '/Restaurant_Review-g187497-d4049034-Reviews-BelleBuon-Barcelona_Catalonia.html', {type: 'RESTAURANTS',element: this,index: 1,section: 0,locationId: '4049034',parentId: '187497',elementType: 'title',selectedId: '4049034'});""><span>BelleBuon</span></div>"
2,Restaurant Ciudad Condal,"Rambla de Catalunya 18, Barcelona, Cataluña, España","<div class=""result-title"" onclick=""widgetEvCall('handlers.openResult', event, this, '/Restaurant_Review-g187497-d15091420-Reviews-Restaurant_Ciudad_Condal-Barcelona_Catalonia.html', {type: 'RESTAURANTS',element: this,index: 2,section: 0,locationId: '15091420',parentId: '187497',elementType: 'title',selectedId: '15091420'});""><span>Restaurant Ciudad Condal</span></div>"
3,Vicky Barcelona,"Dob utca 16., Budapest, Hungría Central, Hungría","<div class=""result-title"" onclick=""widgetEvCall('handlers.openResult', event, this, '/Restaurant_Review-g274887-d9459214-Reviews-Vicky_Barcelona-Budapest_Central_Hungary.html', {type: 'RESTAURANTS',element: this,index: 3,section: 0,locationId: '9459214',parentId: '274887',elementType: 'title',selectedId: '9459214'});""><span>Vicky Barcelona</span></div>"
4,Le Romane,"Carrer Muntaner 103, Barcelona, Cataluña, España","<div class=""result-title"" onclick=""widgetEvCall('handlers.openResult', event, this, '/Restaurant_Review-g187497-d15589100-Reviews-Le_Romane-Barcelona_Catalonia.html', {type: 'RESTAURANTS',element: this,index: 4,section: 0,locationId: '15589100',parentId: '187497',elementType: 'title',selectedId: '15589100'});""><span>Le Romane</span></div>"
...,...,...,...
925,Pacha Barcelona Restaurant,"Paseo Maritimo Barceloneta, 38, Barcelona, Cataluña, España","<div class=""result-title"" onclick=""widgetEvCall('handlers.openResult', event, this, '/Restaurant_Review-g187497-d7081904-Reviews-Pacha_Barcelona_Restaurant-Barcelona_Catalonia.html', {type: 'RESTAURANTS',element: this,index: 25,section: 0,locationId: '7081904',parentId: '187497',elementType: 'title',selectedId: '7081904'});""><span>Pacha Barcelona Restaurant</span></div>"
926,BrewDog Barcelona,"Calle Casanova, 69, Barcelona, Cataluña, España","<div class=""result-title"" onclick=""widgetEvCall('handlers.openResult', event, this, '/Restaurant_Review-g187497-d7610881-Reviews-BrewDog_Barcelona-Barcelona_Catalonia.html', {type: 'RESTAURANTS',element: this,index: 26,section: 0,locationId: '7610881',parentId: '187497',elementType: 'title',selectedId: '7610881'});""><span>BrewDog Barcelona</span></div>"
927,Segafredo Espresso Barcelona,"Passeig del Mare Nostrum 19-21, Barcelona, Cataluña, España","<div class=""result-title"" onclick=""widgetEvCall('handlers.openResult', event, this, '/Restaurant_Review-g187497-d3292563-Reviews-Segafredo_Espresso_Barcelona-Barcelona_Catalonia.html', {type: 'RESTAURANTS',element: this,index: 27,section: 0,locationId: '3292563',parentId: '187497',elementType: 'title',selectedId: '3292563'});""><span>Segafredo Espresso Barcelona</span></div>"
928,Club de L'empanada,"C/ Dagueria 7, Barcelona, Cataluña, España","<div class=""result-title"" onclick=""widgetEvCall('handlers.openResult', event, this, '/Restaurant_Review-g187497-d4072374-Reviews-Club_de_L_empanada-Barcelona_Catalonia.html', {type: 'RESTAURANTS',element: this,index: 28,section: 0,locationId: '4072374',parentId: '187497',elementType: 'title',selectedId: '4072374'});""><span>Club de L'empanada</span></div>"


In [ ]:
print(df_name_and_address.columns)


Index(['Nombre', 'Dirección', 'url'], dtype='object')


In [ ]:
import re

# Función para extraer la dirección web de un fragmento HTML
def extraer_direccion_web(html):
    match = re.search(r"'/Restaurant_Review[^']+'", html)
    if match:
        return match.group(0)
    else:
        return None

df_name_and_address['url2'] = df_name_and_address['url'].apply(extraer_direccion_web)

df_name_and_address = df_name_and_address.drop(columns=['url'])

In [ ]:
df_name_and_address.head(11)

,Nombre,Dirección,Direccion_Web
0,Barcelona Tapas,"Calle Matamoros 906, Puerto Vallarta, México",/Restaurant_Review-g150793-d1032982-Reviews-Barcelona_Tapas-Puerto_Vallarta.html
1,BelleBuon,"Travessera de Gràcia 441, Barcelona, Cataluña, España",/Restaurant_Review-g187497-d4049034-Reviews-BelleBuon-Barcelona_Catalonia.html
2,Restaurant Ciudad Condal,"Rambla de Catalunya 18, Barcelona, Cataluña, España",/Restaurant_Review-g187497-d15091420-Reviews-Restaurant_Ciudad_Condal-Barcelona_Catalonia.html
3,Vicky Barcelona,"Dob utca 16., Budapest, Hungría Central, Hungría",/Restaurant_Review-g274887-d9459214-Reviews-Vicky_Barcelona-Budapest_Central_Hungary.html
4,Le Romane,"Carrer Muntaner 103, Barcelona, Cataluña, España",/Restaurant_Review-g187497-d15589100-Reviews-Le_Romane-Barcelona_Catalonia.html
5,Pulpería Can Lampazas,"Avinguda del Paral·lel, 159, Barcelona, Cataluña, España",/Restaurant_Review-g187497-d10643938-Reviews-Pulperia_Can_Lampazas-Barcelona_Catalonia.html
6,Corchos Barcelona -Colina,"Av boyaca 145c 2, Bogotá, Colombia",/Restaurant_Review-g294074-d12991509-Reviews-Corchos_Barcelona_Colina-Bogota.html
7,El Nacional,"Paseo de Gracia 24, bis, Barcelona, Cataluña, España",/Restaurant_Review-g187497-d7175242-Reviews-El_Nacional-Barcelona_Catalonia.html
8,Can Paixano (La Xampanyeria),"Carreer de la Reina Christina 7, Barcelona, Cataluña, España",/Restaurant_Review-g187497-d940427-Reviews-Can_Paixano_La_Xampanyeria-Barcelona_Catalonia.html
9,Bar Tomas,"Major de Sarrià 49, Barcelona, Cataluña, España",/Restaurant_Review-g187497-d945538-Reviews-Bar_Tomas-Barcelona_Catalonia.html


In [ ]:
df_name_and_address['Direccion_Web'] = df_name_and_address['url2'].str.strip("'")

df_name_and_address = df_name_and_address.drop(columns=['url2'])

df_name_and_address

,Nombre,Dirección,Direccion_Web
0,Barcelona Tapas,"Calle Matamoros 906, Puerto Vallarta, México",/Restaurant_Review-g150793-d1032982-Reviews-Barcelona_Tapas-Puerto_Vallarta.html
1,BelleBuon,"Travessera de Gràcia 441, Barcelona, Cataluña, España",/Restaurant_Review-g187497-d4049034-Reviews-BelleBuon-Barcelona_Catalonia.html
2,Restaurant Ciudad Condal,"Rambla de Catalunya 18, Barcelona, Cataluña, España",/Restaurant_Review-g187497-d15091420-Reviews-Restaurant_Ciudad_Condal-Barcelona_Catalonia.html
3,Vicky Barcelona,"Dob utca 16., Budapest, Hungría Central, Hungría",/Restaurant_Review-g274887-d9459214-Reviews-Vicky_Barcelona-Budapest_Central_Hungary.html
4,Le Romane,"Carrer Muntaner 103, Barcelona, Cataluña, España",/Restaurant_Review-g187497-d15589100-Reviews-Le_Romane-Barcelona_Catalonia.html
...,...,...,...
925,Pacha Barcelona Restaurant,"Paseo Maritimo Barceloneta, 38, Barcelona, Cataluña, España",/Restaurant_Review-g187497-d7081904-Reviews-Pacha_Barcelona_Restaurant-Barcelona_Catalonia.html
926,BrewDog Barcelona,"Calle Casanova, 69, Barcelona, Cataluña, España",/Restaurant_Review-g187497-d7610881-Reviews-BrewDog_Barcelona-Barcelona_Catalonia.html
927,Segafredo Espresso Barcelona,"Passeig del Mare Nostrum 19-21, Barcelona, Cataluña, España",/Restaurant_Review-g187497-d3292563-Reviews-Segafredo_Espresso_Barcelona-Barcelona_Catalonia.html
928,Club de L'empanada,"C/ Dagueria 7, Barcelona, Cataluña, España",/Restaurant_Review-g187497-d4072374-Reviews-Club_de_L_empanada-Barcelona_Catalonia.html


In [305]:
df_name_and_address.to_csv('df_info.csv', index=False)

In [ ]:
df_name_and_address.columns

Index(['Nombre', 'Dirección', 'Direccion_Web'], dtype='object')

In [3]:
df_name_and_address = pd.read_csv("df_info.csv")
df_name_and_address.head()

,Nombre,Dirección,Direccion_Web
0,Barcelona Tapas,"Calle Matamoros 906, Puerto Vallarta, México",/Restaurant_Review-g150793-d1032982-Reviews-Ba...
1,BelleBuon,"Travessera de Gràcia 441, Barcelona, Cataluña,...",/Restaurant_Review-g187497-d4049034-Reviews-Be...
2,Restaurant Ciudad Condal,"Rambla de Catalunya 18, Barcelona, Cataluña, E...",/Restaurant_Review-g187497-d15091420-Reviews-R...
3,Vicky Barcelona,"Dob utca 16., Budapest, Hungría Central, Hungría",/Restaurant_Review-g274887-d9459214-Reviews-Vi...
4,Le Romane,"Carrer Muntaner 103, Barcelona, Cataluña, España",/Restaurant_Review-g187497-d15589100-Reviews-L...


Bucle para obtener Puntajes de cada restaurante dentro de las paginas

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options


opts = Options()
opts.add_argument('-headless=new')
firefox = Firefox(options=opts)
driver = firefox


wait = WebDriverWait(driver, 20)
wait2 = WebDriverWait(driver, 40)

def obtener_puntaje_pagina(driver, nombre_restaurante):
    puntaje_pagina = []

    # Esperar a que el contenedor de puntuación esté presente
    try:
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'QEQvp')))
        wait2.until(EC.presence_of_element_located((By.CLASS_NAME, 'ZDEqb')))
        # Localizar las reseñas dentro del contenedor
        puntaje = driver.find_element(By.CLASS_NAME, 'ZDEqb').text.strip()
        puntaje_pagina.append({'Restaurante': nombre_restaurante, 'Puntaje': puntaje})
        print(f'Puntuación obtenida: {puntaje}')
    except Exception as e:
        print(f'Error al obtener la reseña: {e}')
        return None  # Retorna None en caso de error

    return puntaje_pagina

# Luego en obtener_puntuaciones
def obtener_puntuaciones(driver, df_name_and_address):
    puntajes_totales = []

    for index, row in df_name_and_address.iloc[0:929].iterrows():
        nombre_restaurante = row['Nombre']
        direccion_web = row['Direccion_Web']
        url_tripadvisor = 'https://www.tripadvisor.es' + direccion_web
        driver.get(url_tripadvisor)
        time.sleep(10)
        html_txt = driver.page_source
        soup = BeautifulSoup(html_txt, 'html.parser')
        puntuacion_pagina = obtener_puntaje_pagina(driver, nombre_restaurante)

        if puntuacion_pagina is not None:
            puntajes_totales.extend(puntuacion_pagina)

    columnas = ['Restaurante', 'Puntaje']
    df_puntajes = pd.DataFrame(puntajes_totales, columns=columnas)

    return df_puntajes


# Luego llamar a la función
df_puntuaciones_final = obtener_puntuaciones(driver, df_name_and_address)

In [294]:
df_puntuaciones_final

,Restaurante,Puntaje
0,Barcelona Tapas,"4,5"
1,BelleBuon,"5,0"
2,Restaurant Ciudad Condal,"4,5"
3,Vicky Barcelona,"4,5"
4,Le Romane,"5,0"
...,...,...
924,Restaurante Sannin,"4,5"
925,Pacha Barcelona Restaurant,"3,5"
926,BrewDog Barcelona,"4,0"
927,Segafredo Espresso Barcelona,"2,5"


In [295]:
df_puntuaciones_final.to_csv('df_puntuaciones.csv', index=False)

In [7]:
import pandas as pd
df_info = pd.read_csv("df_info.csv")
df_info.head()


,Nombre,Dirección,Direccion_Web
0,Barcelona Tapas,"Calle Matamoros 906, Puerto Vallarta, México",/Restaurant_Review-g150793-d1032982-Reviews-Ba...
1,BelleBuon,"Travessera de Gràcia 441, Barcelona, Cataluña,...",/Restaurant_Review-g187497-d4049034-Reviews-Be...
2,Restaurant Ciudad Condal,"Rambla de Catalunya 18, Barcelona, Cataluña, E...",/Restaurant_Review-g187497-d15091420-Reviews-R...
3,Vicky Barcelona,"Dob utca 16., Budapest, Hungría Central, Hungría",/Restaurant_Review-g274887-d9459214-Reviews-Vi...
4,Le Romane,"Carrer Muntaner 103, Barcelona, Cataluña, España",/Restaurant_Review-g187497-d15589100-Reviews-L...


In [8]:
df_info.shape

(930, 3)

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options
import re


opts = Options()
opts.add_argument('-headless=new')
firefox = Firefox(options=opts)
driver = firefox


wait = WebDriverWait(driver, 20)
wait2 = WebDriverWait(driver, 40)

def obtener_reseñas_pagina(driver, nombre_restaurante):
    reseñas_pagina = []
  
    # Esperar a que el contenedor de reseñas esté presente
    try:
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'listContainer')))
    except Exception as e:
        print(f'Error al esperar el contenedor de reseñas: {e}')
        return reseñas_pagina

    # Localizar el contenedor de reseñas
    reviews_container = driver.find_element(By.CLASS_NAME, 'listContainer')

    # Localizar las reseñas dentro del contenedor
    review_containers = reviews_container.find_elements(By.CLASS_NAME, 'review-container')

    for i, review_container in enumerate(review_containers, start=1):
        try:
            # Volver a localizar los elementos en cada iteración
            rating_class = review_container.find_element(By.CLASS_NAME, 'ui_bubble_rating').get_attribute('class')
            bubles = re.search(r'\d+', rating_class).group()

            print("bubles colected")
            reseña = review_container.find_element(By.CLASS_NAME, 'partial_entry').text.strip()
            print("reseña colected")
            # Agregar la reseña a la lista
            reseñas_pagina.append({'Restaurante': nombre_restaurante, 'Reseña': reseña, 'Bubles': bubles})
        except Exception as e:
            print(f'Error al obtener la reseña {i}: {e}')

    return reseñas_pagina

def obtener_reseñas(driver, df_info):
    reseñas_totales = []

    for index, row in df_info.iloc[1:825].iterrows():

        nombre_restaurante = row['Nombre']
        direccion_web = row['Direccion_Web']
        url_tripadvisor = 'https://www.tripadvisor.es' + direccion_web

        # Accede a la página del restaurante
        driver.get(url_tripadvisor)
        time.sleep(10)
         
        # Extraer datos, convertir html a texto
        html_txt = driver.page_source
        soup = BeautifulSoup(html_txt, 'html.parser')

        try:
            wait.until(EC.presence_of_element_located((By.ID, 'filters_detail_language_filterLang_en')))
        except Exception as e:
            print(f'Error al esperar el checkmark de idioma: {e}')
    
    
        checkmark = driver.find_element(By.XPATH, "//label[@class='label container' and @for='filters_detail_language_filterLang_en']")
        checkmark.click()   

        # Esperar a que el botón "Más" esté presente y sea cliclable
        try:
            wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'p.partial_entry span.taLnk')))
        except Exception as e:
            print(f'Error al esperar el botón "Más": {e}')
        try:
        # Encontrar el botón por su selector CSS
            boton_mas = driver.find_element(By.CSS_SELECTOR, 'p.partial_entry span.taLnk')

        # Hacer clic en el botón "Más"
            boton_mas.click()
        except NoSuchElementException:
           print('El botón "Más" no fue encontrado en esta página.')
            
        # Obtener reseñas de la página actual
        reseñas_pagina = obtener_reseñas_pagina(driver, nombre_restaurante)

        # Imprimir o almacenar las reseñas según sea necesario
        reseñas_totales.extend(reseñas_pagina)

        # Hacer clic en el botón siguiente para obtener más reseñas
        try:
            max_paginas = 3 # Ajusta el límite según sea necesario

            for pagina in range(1, max_paginas + 1):
                time.sleep(30)
                try:
                    # Esperar a que el enlace de la página esté presente y sea cliclable
                    enlace_pagina = wait.until(EC.element_to_be_clickable((By.XPATH, f'//a[@data-page-number="{pagina}"]')))
                    # Hacer clic en el enlace de la página
                    enlace_pagina.click()

                    # Esperar a que la siguiente página cargue completamente
                    wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'listContainer')))
                except Exception as e:
                    print(f'Error durante la iteración de la página {pagina}: {e}')

                # Obtener reseñas de la página actual
                reseñas_pagina = obtener_reseñas_pagina(driver, nombre_restaurante)

                # Imprimir o almacenar las reseñas según sea necesario
                reseñas_totales.extend(reseñas_pagina)

        except Exception as e:
            print(f'Error durante el scraping: {e}')

    # Convertir la lista de reseñas a un DataFrame al final
    columnas = ['Restaurante', 'Reseña', 'Bubles']
    df_reseñas = pd.DataFrame(reseñas_totales, columns=columnas)

    return df_reseñas

# Luego llamar a la función
df_reseñas_final= obtener_reseñas(driver, df_info)

In [58]:
df_reseñas_final

,Restaurante,Reseña,Bubles
0,Bar Rodrigo,We ordered a paella mixed for two. The pan was...,10
1,Bar Rodrigo,"From the moment I walked into Bar Rodrigo, I f...",50
2,Bar Rodrigo,"Waitstaff were friendly, food was amazing (we ...",50
3,Bar Rodrigo,The owner was a horrible man. No eye contact w...,10
4,Bar Rodrigo,This is the most expensive and worst meal I’ve...,30
...,...,...,...
7335,el tianguis,"it s a tiny restaurant with mexican food, very...",50
7336,el tianguis,I would definetly recommend this mexican resta...,50
7337,el tianguis,"Excelentes tacos, como en Mexico, nothing more...",50
7338,el tianguis,We went there looking for some mexican snacks....,40
